In [1]:
%config IPCompleter.greedy=True  # Podpowiedzi składni
import numpy as np
from scipy.spatial import distance_matrix
import matplotlib.pyplot as plt
import random
import itertools
import math
import copy
import time
from sortedcontainers import SortedList

# Własna funkcja zaokrąglająca
def round2(number):
    if (number - math.trunc(number) >= 0.5):
        return math.trunc(number) + 1
    else:
        return math.trunc(number)

In [2]:
file = open('kroA200.tsp.txt')
# file = open('kroB200.tsp.txt')

In [3]:
# Pomiń zbędne linie

next_line = file.readline()[0:-1]
while next_line[0] != "1":
    next_line = file.readline()[0:-1]
print(next_line)

1 1357 1905


In [4]:
# Wczytaj wszystkie punkty do tablicy

points_tab = []
while next_line!= "EOF":
    points_tab.append(next_line.split(" "))
    next_line = file.readline()[0:-1]
    
for x in range(len(points_tab)):
    for y in range(len(points_tab[0])):
        points_tab[x][y] = int(points_tab[x][y])
    points_tab[x][0] = points_tab[x][0] - 1  # zmniejsz identyfikatory punktów o 1
print(len(points_tab))
print(len(points_tab[0]))


#print(points_tab)

200
3


In [5]:
# Utwórz tablicę odległości między punktami

length_tab = list(np.zeros((len(points_tab),len(points_tab)), dtype=int))
for x in range(len(length_tab)):
    for y in range(len(length_tab[0])):
        length_tab[x][y] = round2(math.sqrt((points_tab[x][1] - points_tab[y][1])**2 + (points_tab[x][2] - points_tab[y][2])**2))  # długość między punktem "x" i "y" w tablicy base_tab

        
#print(length_tab)

In [6]:
# Funkcja zwracająca długość ścieżki

def distance(path):
    dist = 0
    for x in range(len(path) - 1):
        dist+=length_tab[path[x]][path[x+1]]
    dist+=length_tab[path[len(path)-1]][path[0]]
    return dist

In [7]:
# funkcja zwracająca losową ścieżkę

def get_random_path(data_size = 200):
    path = np.arange(data_size)
    np.random.shuffle(path)
    inside = list(path[:int(data_size/2)])
    outside = list(path[int(data_size/2):])
    return inside, outside

In [8]:
# Funkcja rysująca punkty na płaszczyźnie - zależnie od pochodzenia odpowiednio w różnych kolorach, połączone lub nie

def print_graph(path, out):
    points_inside = []
    points_outside = []

    plt.figure()
    for i in path:
        points_inside.append(copy.deepcopy(points_tab[i][1:]))
    for i in out:
        points_outside.append(copy.deepcopy(points_tab[i][1:]))

    points_inside_0 = []
    for x in points_inside:
        points_inside_0.append(copy.deepcopy(x[0]))
    points_inside_0.append(copy.deepcopy(points_inside[0][0]))
    
    points_inside_1 = []
    for x in points_inside:
        points_inside_1.append(copy.deepcopy(x[1]))
    points_inside_1.append(copy.deepcopy(points_inside[0][1]))

    points_outside_0 = []
    for x in points_outside:
        points_outside_0.append(copy.deepcopy(x[0]))
    points_outside_1 = []
    for x in points_outside:
        points_outside_1.append(copy.deepcopy(x[1]))

    plt.plot(points_inside_0,points_inside_1, '-o')
    plt.plot(points_outside_0,points_outside_1, 'o')
    plt.show()

    
#print_graph(path, out)

In [9]:
# Funkcje pomocnicze

def p_dist(a, b):  
    return length_tab[a][b]

def out_of(a, size = 100):  
    while a < 0:
        a = size + a
    while a >= size:
        a = a - size
    return a

In [10]:
# Funkcje algorytmu steepest

def give_best_edges(path):
    best_imp1 = 0
    best_edges = []

    for x in range(len(path))[:]:  # Dla każdej krawędzi
        for y in range(len(path))[:]:  # Dla każdej innej krawędzi
            if x!=y:

                new_imp = p_dist(path[x], path[out_of(x+1)]) + p_dist(path[y], path[out_of(y+1)]) - p_dist(path[x], path[y]) - p_dist(path[out_of(x+1)], path[out_of(y+1)])
                if new_imp > best_imp1:  # Jeśli jest to lepsze rozwiązanie, zapamiętaj je
                    best_imp1 = new_imp
                    best_edges = [x, y]
    
    return best_imp1, best_edges


def give_best_vertices(path, out):
    best_imp2 = 0
    best_vertices = []
    
    for x in range(len(path))[:]:  # Dla każdego wierzchołka ze ścieżki
            for y in range(len(out))[:]:  # Dla każdego wierzchołka spoza ścieżki

                new_imp = p_dist(path[out_of(x-1)], path[x]) + p_dist(path[x], path[out_of(x+1)]) - p_dist(path[out_of(x-1)], out[y]) - p_dist(out[y], path[out_of(x+1)])
                if new_imp > best_imp2:  # Jeśli jest to lepsze rozwiązanie, zapisz zmiany
                    best_imp2 = new_imp
                    best_vertices = [x, y]
                    
    return best_imp2, best_vertices


In [11]:
# Funkcja zmieniająca ścieżkę zgodnie z podanymi danymi

def change_your_path(path, out, best_imp1, best_imp2, best_edges, best_vertices):
    if best_imp1 >= best_imp2:
        #podmień krawędzie
        x = best_edges[0]
        y = best_edges[1]

        for z in range(int(abs(y - x)/2)):  # Zamiana krawędzi, czyli obrócenie kolejności (krawędź1; krawędź2>
            if x < y:
                path[x + 1 + z], path[y - z] = path[y - z], path[x + 1 + z]
            else:
                path[y + 1 + z], path[x - z] = path[x - z], path[y + 1 + z]
    else:
        #podmień punkty
        x = path[best_vertices[0]]
        y = out[best_vertices[1]]

        path.insert(path.index(x), y)
        path.remove(x)

        out.append(x)
        out.remove(y)

In [12]:
# Funkcja standaryzująca ścieżkę - rozpoczynamy od najmniejszego elementu, zachowując kolejność. Ułatwia porównywanie.

def list_organizer(tab):
    min_index = tab.index(min(tab))

    new_tab = [] 
    for x in range(len(tab)):
        new_tab.append(tab[out_of(min_index + x, len(tab))])  # Funkcja out_of automatycznie zawija zbyt duże indeksy
    return new_tab

In [13]:
# Funkcja sprawdzająca, czy dana scieżka NIE występuje w populacji. Korzysta z pracy funkcji <list_organizer>.

def not_in_population(population, new_member):
    for x in range(len(population)):
        if population[x] == new_member:
            return False
    return True

In [14]:
def give_reversed(tab):
    new_tab = copy.deepcopy(tab)
    for x in range(int(len(tab)/2)):
        new_tab[x], new_tab[len(tab)-1-x] = tab[len(tab)-1-x], tab[x]
    return new_tab

In [15]:
# Liczba powtórzeń testowych

test_repeat = 10
MSLS_AVG = 834.49659276

# Evolutionary Hybrid Algorithm

In [16]:
# Algorytm główny

'''
min, max i średnia długość ścieżki i czas
najlepsze rozwiązanie jako path i out
'''
min_path = 1000000
average_path = 0
max_path = 0

min_time = time.perf_counter() + 100000
average_time = 0
max_time = 0

best_path = [0]
best_out = [0]



for alfa in range(test_repeat):  # powtórz testy n razy
    
    evol_time = time.perf_counter()
    
    # Utwórz populację początkową
    population = []
    while len(population) < 20:
        path, out = get_random_path(len(points_tab))
        path = list_organizer(path)
        if not_in_population(population, path):
            population.append([path, out, distance(path)])

    
    while(time.perf_counter() - evol_time < MSLS_AVG):


        # Wylosuj 2 rodziców
        parent1 = random.randint(0, 19)

        parent2 = random.randint(0, 18)
        if parent2 >= parent1:  # Matematyczna sztuczka z losowaniem liczby z przedziału (0;19) POZA liczbą parent1
            parent2 += 1

        parent1, out1 = copy.deepcopy(population[parent1][0]), copy.deepcopy(population[parent1][1])
        parent2, out2 = copy.deepcopy(population[parent2][0]), copy.deepcopy(population[parent2][1])


        # Wyszukaj wszystkie wspólne podścieżki
        all_subpaths = []  # znalezione ścieżki
        all_vertices = []  # wszystkie punkty ze znalezionych ścieżek

        # Rodzice w kolejności identycznej
        for x in range(len(parent1)):

            if parent1[x] in parent2 and parent1[x] not in all_vertices:
                new_subpath = [copy.deepcopy(parent1[x])]  # aktualnie tworzona ścieżka
                y = parent2.index(parent1[x])

                z = 1
                while parent1[out_of(x + z)] == parent2[out_of(y + z)]:
                    new_subpath.append(copy.deepcopy(parent1[out_of(x + z)]))
                    z+=1

                z = 1
                while parent1[out_of(x - z)] == parent2[out_of(y - z)]:
                    new_subpath.insert(0, parent1[out_of(x - z)])
                    z+=1
                    
                if len(new_subpath) > 1:  # żeby reversed miał punkty, które nie są jeszcze w vertices
                    all_subpaths.append(new_subpath)
                    all_vertices.extend(new_subpath)


        # Rodzice w kolejności reversed (parent2 obrócone, ścieżki identyczne logicznie lecz odwrotnie zapisane)
        reversed_parent2 = copy.deepcopy(give_reversed(parent2))
        for x in range(len(parent1)):

            if parent1[x] in reversed_parent2 and parent1[x] not in all_vertices:
                new_subpath = [parent1[x]]  # aktualnie tworzona ścieżka
                y = reversed_parent2.index(parent1[x])

                z = 1
                while parent1[out_of(x + z)] == reversed_parent2[out_of(y + z)]:
                    new_subpath.append(parent1[out_of(x + z)])
                    z+=1

                z = 1
                while parent1[out_of(x - z)] == reversed_parent2[out_of(y - z)]:
                    new_subpath.insert(0, parent1[out_of(x - z)])
                    z+=1

                all_subpaths.append(new_subpath)
                all_vertices.extend(new_subpath)


        # Dopóki brakuje punktów, dolosowuj je!
        if len(all_vertices) < 100:

            # Lista dostępnych punktów
            other_vertices = copy.deepcopy(parent1)
            other_vertices.extend(copy.deepcopy(parent2))

            for x in other_vertices[:]:
                if x in all_vertices:
                    other_vertices.remove(x)


            # Dolosuj nowe punkty
            while len(all_vertices) < 100:
                new_vertice_index = random.randint(0, len(other_vertices) - 1)  # los z CAŁEGO zakresu listy other_vertices

                all_subpaths.append([other_vertices[new_vertice_index]])
                all_vertices.append(other_vertices[new_vertice_index])
                other_vertices.remove(other_vertices[new_vertice_index])


        # Teraz utwórz child
        child = []
        for x in range(len(all_subpaths))[:]:

            subpath_index = random.randint(0, len(all_subpaths) - 1)  # losujemy podścieżkę do wstawienia
            direction = random.randint(0, 1)  # losujemy kierunek podścieżki

            if direction == 0:
                child.extend(copy.deepcopy(all_subpaths[subpath_index]))
            else:
                child.extend(copy.deepcopy(give_reversed(all_subpaths[subpath_index])))

            all_subpaths.remove(all_subpaths[subpath_index])


        # Utwórz listę wierzchołków spoza ścieżki child
        child_out = [x for x in range(200)]
        for x in child_out[:]:
            if x in child:
                child_out.remove(x)


        # Algorytm steepest
        while(True):
            best_imp1, best_edges = give_best_edges(child)
            best_imp2, best_vertices = give_best_vertices(child, child_out)

            # Jeśli znaleziono jakąś poprawę: zapisz rezultat
            if best_imp1 > 0 or best_imp2 > 0:  
                change_your_path(child, child_out, best_imp1, best_imp2, best_edges, best_vertices)    
            # Warunek stopu
            else:
                break


        # Jeśli jest lepsze od najgorszego i inne od każdego rozwiązania w populacji - dodaj i usuń najgorsze
        child_dist = distance(child)

        worst_dist = population[0][2]
        worst_index = 0
        for x in range(len(population)):
            if population[x][2] > worst_dist:  # Jeśli długość jest jeszcze większa
                worst_dist = population[x][2]
                worst_index = x


        if child_dist < worst_dist:  # Jeśli child jest lepszym rozwiązaniem od najgorszego z populacji
            child = list_organizer(child)

            if not_in_population(population, child):  # Jeśli jest unikalne
                population.remove(population[worst_index])
                population.append([child, child_out, child_dist])

        
        
                
    evol_time = time.perf_counter() - evol_time
    
    # Znajdź najlepsze rozwiązanie
    best_dist = 1000000
    best_index = 0
    for x in range(len(population)):
        if population[x][2] < best_dist:
            best_dist = population[x][2]
            best_index = x
                 
    
    if population[best_index][2] < min_path:
        min_path = population[best_index][2]
        best_path = population[best_index][0]
        best_out = population[best_index][1]
    if population[best_index][2] > max_path:
        max_path = population[best_index][2]
    average_path += population[best_index][2]
    
    
    if evol_time < min_time:
        min_time = evol_time
    if evol_time > max_time:
        max_time = evol_time
    average_time += evol_time
    

print("Min_path: "+str(min_path))
average_path = average_path/test_repeat
print("Average_path: "+str(average_path))
print("Max_path: "+str(max_path))
print()
print("Min_time: "+str(min_time))
average_time = average_time/test_repeat
print("Average_time: "+str(average_time))
print("Max_time: "+str(max_time))



print("\nBEST: ", end = '')
print(distance(best_path))  # Najkrótsza ścieżka
print_graph(best_path, best_out)





KeyboardInterrupt: 